In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

### Decision Tree Classifier
![Decision Tree.png](https://raw.githubusercontent.com/vaibhaVishwakarma/ML-from-scratch/master/NoteBooks/Decision%20Tree.png)

In [2]:
class Node:
    def __init__(self , feature = None , threshold=None ,left = None , right = None , * , value = None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    def is_leaf(self):
        return self.value is not None
    

In [3]:
class DecisionTreeClassifer:
    # This tree is limited to - 2 child per node 
    
    def __init__(self , min_samples_split = 2 , max_dept = 100 , n_features = None):
        self.min_samples_split = min_samples_split
        self.max_dept = max_dept
        self.n_features = n_features
        self.root = None
    
    def fit(self , X , y):
        if(y.dtype == np.float_):
            raise Exception("This implementation only supports Classification.\nPassed target value is Numerical/Regression")

        _ , self.n_features = X.shape
        self.root = self._grow_tree(X,y)
    
    def _grow_tree(self, X , y , depth = 0):
        n_samples , n_features = X.shape
        n_lables = len(np.unique(y))

        #check stopping criteria
        if (depth >= self.max_dept or n_lables == 1  or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value = leaf_value)
        

        #revlevance with Random Forest, no use in simple tree.
        feature_idxs = np.random.choice(n_features , self.n_features , replace=False)  #retures specified numbers of features (default = 1) only without repitition

        best_feature , best_threshold = self._best_split(X , y , feature_idxs)

        #create Child nodes and row level spliting
        left_records_idxs , right_records_idxs = self._split(X[: , best_feature] , best_threshold)

        if len(left_records_idxs) == 0 or len(right_records_idxs) == 0 :
            leaf_value = self._most_common_label(y)
            return Node(value = leaf_value)


        left =  self._grow_tree( X[left_records_idxs , :  ] , y[left_records_idxs], depth+1) 
        right = self._grow_tree( X[right_records_idxs , : ] , y[right_records_idxs], depth+1)

        return Node(feature = best_feature,
                    threshold = best_threshold,
                    left = left,
                    right = right)     
    
    def _best_split(self , X , y , feature_idxs):
        best_gain = -1
        split_idx  , split_threshold = 0 , 0

        #iterate each feature to calculate the max values of these attributes
        for feature_idx in feature_idxs:
            feature_column = X[:, feature_idx]
            thresholds = np.unique(feature_column)

            for threshold in thresholds:
                #get information gain
                gain = self._information_gain(y , feature_column , threshold)

                if gain > best_gain:
                    best_gain = gain 
                    split_idx = feature_idx
                    split_threshold = threshold

        return split_idx , split_threshold
        
    def _information_gain(self, y , feature_column , threshold):
        parent_entropy = self._entropy(y)

        left_records_idxs , right_records_idxs = self._split(feature_column , threshold)

        if len(left_records_idxs) == 0 or len(right_records_idxs) == 0 :
            return 0

        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l , n_r = len(left_records_idxs) , len(right_records_idxs)
        e_l , e_r = self._entropy(y[left_records_idxs]) , self._entropy(y[right_records_idxs])
        child_entropy = (n_l/n)*e_l + (n_r/n)*e_r

        information_gain = parent_entropy - child_entropy
        return information_gain

    def _entropy(self, y):
        hist = np.bincount(y)
        probability_set = hist / len(y)
        return - np.sum([ ( probability * np.log(probability) ) for probability in probability_set])
    
    def _split(self, column , threshold):
        left_rows = np.argwhere(column <= threshold).flatten() #[[1],[2]] -> [1 ,2]
        right_rows = np.argwhere(column > threshold).flatten()
        return left_rows , right_rows

    def _most_common_label(self , y):
        return Counter(y).most_common(1)[0][0] #gets most common key from [[val , freq]]
    
    def predict(self , X):
        return  np.array([self._traverse_tree(record, self.root) for record in X])
    
    def _traverse_tree(self, x , node):
        if node.is_leaf():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x , node.left)
        return self._traverse_tree(x , node.right)
    

In [4]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier as sk_DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [10]:
datasets_  = [datasets.load_breast_cancer() , datasets.load_wine() , datasets.load_iris()]
for data in datasets_:
    X , y = data.data , data.target
    
    X_train , X_test , y_train , y_test = train_test_split(X , y , test_size= 0.2 , shuffle = True , random_state=4444)
    dt , sk_dt = DecisionTreeClassifer() , sk_DecisionTreeClassifier()

    dt.fit(X_train , y_train)
    sk_dt.fit(X_train , y_train)

    print(f"custom  Decision Tree accuracy: {100*accuracy_score(y_test , dt.predict(X_test))}")
    print(f"sklearn Decision Tree accuracy: {100*accuracy_score(y_test , sk_dt.predict(X_test))}\n")

custom  Decision Tree accuracy: 95.6140350877193
sklearn Decision Tree accuracy: 93.85964912280701

custom  Decision Tree accuracy: 69.44444444444444
sklearn Decision Tree accuracy: 94.44444444444444

custom  Decision Tree accuracy: 50.0
sklearn Decision Tree accuracy: 90.0



#### Rough

In [6]:
np.array([1,2,3,4,5])[[2,3]]

array([3, 4])

In [7]:
np.isnan(X).sum(axis=0)

array([0, 0, 0, 0])

In [8]:
Counter([]).most_common(1)[0]

IndexError: list index out of range

In [ ]:
l , r  = dt._split(X[0,:] , 1.2)
l,r

(array([ 4,  5,  6,  7,  8,  9, 10, 11, 14, 15, 16, 17, 18, 19, 24, 25, 26,
        27, 28, 29], dtype=int64),
 array([ 0,  1,  2,  3, 12, 13, 20, 21, 22, 23], dtype=int64))

In [ ]:
np.bincount([1,2,3,4,5,6,7])

In [ ]:
np.random.choice(range(100,200),10,replace=True)


array([161, 161, 149, 173, 172, 115, 162, 117, 164, 129])

In [ ]:
np.argwhere(np.array([float(1.22222),2,3,4,5,6])>1.1).flatten()

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [ ]:
arr = [1,2,3,3,4,4,4,4,4,5]
[i**2 for i in arr if i%2 ==0]

[4, 16, 16, 16, 16, 16]

In [ ]:
from collections import Counter
cnt = Counter(arr)